In [1]:
from ephys_queries import (
    select_spike_times
)
from ephys_queries import db_setup_core
from dotenv import load_dotenv

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
from spiketimes.df.correlate import cross_corr_test

In [4]:
data_dir = Path(".").absolute().parent / "data"
group_names = ("chronic_citalopram", 
                           "chronic_saline", 
                           "chronic_saline_", 
                           "citalopram_continuation", 
                           "citalopram_discontinuation")
dfb = (
    pd.read_csv(data_dir / "baseline.csv")
    .assign(group= 
            lambda x: x["group_name"].map({"chronic_saline": "CS",
                                          "chronic_saline_": "CS",
                                          "citalopram_continuation": "CC",
                                          "chronic_citalopram": "CC",
                                          "citalopram_discontinuation": "CD"})
           )
    .loc[lambda x: x.group_name.isin(group_names)]
)

In [5]:
load_dotenv()
engine, metadata = db_setup_core()
block_name = "pre"
fs = 30000

df_spiketimes = (
    select_spike_times(
        engine, metadata, 
        block_name=block_name, 
        group_names=group_names,
    )
    .assign(
        spiketimes= lambda x: x["spike_time_samples"].divide(fs)
    )
    .drop("spike_time_samples", axis=1)
    .merge(dfb[["cluster", "neuron_id", "session_name", "group_name"]])
    .loc[lambda x: x["cluster"] != "no_baseline"]
)

In [6]:
session_names = df_spiketimes["session_name"].unique()

In [7]:
big_frames = []
BINS = np.array([0.001, 0.01])
NUM_LAGS=200

for i, BIN in enumerate(BINS):
    print(f"{(i/len(BINS) * 100)}%") 
    frames = []
    for session in session_names:
        frames.append(
            df_spiketimes
            .loc[lambda x: x["session_name"]==session]
            .pipe(
                lambda x: cross_corr_test(
                    x,
                    binsize=BIN,
                    num_lags=NUM_LAGS,
                    spiketimes_col="spiketimes", 
                    spiketrain_col="neuron_id",
                    use_multiprocessing=True,
                    max_cores=10
                )
            )
            .assign(session_name=session,
                   num_lags=NUM_LAGS)
        )
    big_frames.append(
        pd.concat(frames).assign(binsize=BIN)
    )
df = pd.concat(big_frames)

0.0%
50.0%


In [8]:
df_done = (
    df
    .merge(dfb[["neuron_id", "cluster"]], left_on="spiketrain_1", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"cluster": "spiketrain_1_cluster"})
    .merge(dfb[["neuron_id", "cluster"]], left_on="spiketrain_2", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"cluster": "spiketrain_2_cluster"})
    .assign(has_sr=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "slow_regular") or (y.spiketrain_2_cluster== "slow_regular"),
                       axis=1),
            has_sir=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "slow_irregular") or (y.spiketrain_2_cluster== "slow_irregular"),
                       axis=1),
            has_ff=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "fast_firing") or (y.spiketrain_2_cluster== "fast_firing"),
                       axis=1)
           )
    .assign(comb= lambda x: x.apply(lambda y: 
                                    "sr_sr" if y.has_sr and (not y.has_sir) and (not y.has_ff)
                                   else "sr_sir" if y.has_sr and y.has_sir and (not y.has_ff)
                                   else "sr_ff" if y.has_sr and (not y.has_sir) and y.has_ff
                                   else "sir_sir" if (not y.has_sr) and y.has_sir and (not y.has_ff)
                                   else "sir_ff" if (not y.has_sr) and y.has_sir and y.has_ff
                                   else "ff_ff", axis=1
                                   ))
)

In [9]:
df_done.to_csv(data_dir / "cross_corr.csv", index=False)

In [10]:
idx = (
    df_done
    .loc[lambda x: abs(x.time_bin) > 0.001]
    .groupby(["spiketrain_1", "spiketrain_2", "binsize"])["p"]
    .idxmin()
).values

In [13]:
(
    df_done
    .loc[lambda x: abs(x.time_bin) > 0.001]
    .groupby(["spiketrain_1", "spiketrain_2", "binsize"])["p"]
    .idxmin()
    .reset_index()
    .drop("p", axis=1)
    .assign(time_bin=df_done.iloc[idx].time_bin.values,
           lowest_p=df_done.iloc[idx].p.values)
    .merge(dfb[["neuron_id", "cluster"]], left_on="spiketrain_1", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"cluster": "spiketrain_1_cluster"})
    .merge(dfb[["neuron_id", "cluster", "group"]], left_on="spiketrain_2", right_on="neuron_id")
    .drop("neuron_id", axis=1)
    .rename(columns={"cluster": "spiketrain_2_cluster"})
    .assign(has_sr=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "slow_regular") or (y.spiketrain_2_cluster== "slow_regular"),
                       axis=1),
            has_sir=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "slow_irregular") or (y.spiketrain_2_cluster== "slow_irregular"),
                       axis=1),
            has_ff=lambda x: 
                x.apply(lambda y: (y.spiketrain_1_cluster == "fast_firing") or (y.spiketrain_2_cluster== "fast_firing"),
                       axis=1)
           )
    .assign(comb= lambda x: x.apply(lambda y: 
                                    "sr_sr" if y.has_sr and (not y.has_sir) and (not y.has_ff)
                                   else "sr_sir" if y.has_sr and y.has_sir and (not y.has_ff)
                                   else "sr_ff" if y.has_sr and (not y.has_sir) and y.has_ff
                                   else "sir_sir" if (not y.has_sr) and y.has_sir and (not y.has_ff)
                                   else "sir_ff" if (not y.has_sr) and y.has_sir and y.has_ff
                                   else "ff_ff", axis=1
                                   ))
    .to_csv(data_dir / "cross_corr_simp.csv", index=False)
)